<a href="https://colab.research.google.com/github/kifoo/Sieci_neuronowe/blob/main/SN_projekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kod implementacji płytkiej sieci neuronowej

In [ ]:
!pip install ucimlrepo
!pip install tensorflow

Importing required libraries

In [2]:
import numpy as np
import pandas as pd
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from ucimlrepo import fetch_ucirepo

Function fetching dataset from https://archive.ics.uci.edu/dataset/602/dry+bean+dataset

In [3]:
def data_fetch():
    # fetch dataset
    dry_bean_dataset = fetch_ucirepo(id=602)

    # data (as pandas dataframes)
    X = dry_bean_dataset.data.features
    Y = dry_bean_dataset.data.targets

    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)
    return X, Y

Function defining the neural network model

In [4]:
# Define the neural network model
def create_model(input_shape, output_shape, nuerons_output, hidden_layers):
    model = Sequential()
    model.add(Dense(nuerons_output, activation='relu', input_shape=(input_shape,)))
    for i in range(hidden_layers):
        model.add(Dense(nuerons_output, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    return model

Creating the neural network model, training it and testing it accuracy

In [5]:
def neural_network(opt_function, loss_function, nuerons_output, hidden_layers, epochs):
    # This function return the dataframe
    X, Y = data_fetch()

    # Split the dataset into a training set and a testing set
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

    # Normalize the inputs
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # print("========== Train ==========")
    # print(y_train.shape[0])
    # print("========== Test ==========")
    # print(X_train.shape[1])

    # Create the neural network model
    model = create_model(input_shape=X_train.shape[1], output_shape=7, nuerons_output=nuerons_output, hidden_layers=hidden_layers)

    # Compile the model
    model.compile(optimizer=opt_function, loss=loss_function, metrics='accuracy')

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

    # Test the model
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")
    return accuracy

Loss function exapmles:

**'mean_squared_error' or 'mse':** Measures the average squared difference between
the true and predicted labels. It is useful for regression problems.

**'binary_crossentropy':** Applies a binary cross-entropy between the true and predicted labels. It is useful for binary classification problems.

**'categorical_crossentropy':** Applies a categorical cross-entropy between the true and predicted labels. It is useful for multi-class classification problems.

**'hinge':** Defines a hinge loss function that measures the difference between the predicted labels and true labels.

**'logcosh':** Defines a logarithm of the hyperbolic cosine loss function.

**'sparse_categorical_crossentropy':** Similar to 'categorical_crossentropy', but it can accept labels with integer values instead of one-hot encoded vectors.

**'kl_divergence':** Calculates the Kullback-Leibler divergence, which quantifies the difference between two probability distributions.

**'poisson':** Calculates the Poisson loss between the true and predicted labels.

In [6]:
def write_data(neuron_outputs,hidden_layers,epochs,acc):
    csv_file_path ="content/data.csv"

    with open(csv_file_path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([neuron_outputs, hidden_layers, epochs, acc])
        print("ok\n")


def main():
    loss = ['mean_squared_error', 'binary_crossentropy', 'categorical_crossentropy', 'hinge', 'logcosh', 'sparse_categorical_crossentropy', 'kl_divergence']
    neuron_outputs = [2, 4, 8, 12, 16]
    hidden_layers = [0, 1, 2]  # góra-cy, ale liczymy od 0 jak prawdziwi informatycy XDD
    epochs = [5, 10, 25, 50]

    # Ej bo tu wychodzi jakieś 60 danych XDD Dokładnie to 63, nie wiem czemu XDD
    min_acc = 1
    min_data = np.array([[0], [0], [0], [0.0]])
    max_acc = 0
    max_data = np.array([[0], [0], [0], [0.0]])
    write_data("Neuron Output","Hidden Layers","Epochs","Accuracy")

    for a in range(len(neuron_outputs)):
          for b in range(len(hidden_layers)):
              for c in range(len(epochs)):
                  acc = neural_network(opt_function='Adam', loss_function=loss[5], nuerons_output=neuron_outputs[a], hidden_layers=hidden_layers[b], epochs=epochs[c])
                  if( acc > max_acc):
                      max_acc = acc
                      max_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  if( acc < min_acc):
                      min_acc = acc
                      min_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  write_data(neuron_outputs[a],hidden_layers[b],epochs[c],acc)

    print(f"Min accuracy: {min_data} \nMax accuracy: {max_data}")

main()

ok



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/5
213/213 [==============================] - 2s 6ms/step - loss: 1.8130 - accuracy: 0.2742 - val_loss: 1.7130 - val_accuracy: 0.2726
Epoch 2/5
213/213 [==============================] - 1s 6ms/step - loss: 1.6167 - accuracy: 0.2722 - val_loss: 1.5136 - val_accuracy: 0.2620
Epoch 3/5
213/213 [==============================] - 1s 5ms/step - loss: 1.3888 - accuracy: 0.3149 - val_loss: 1.2503 - val_accuracy: 0.4262
Epoch 4/5
213/213 [==============================] - 1s 6ms/step - loss: 1.1565 - accuracy: 0.4604 - val_loss: 1.0612 - val_accuracy: 0.6121
Epoch 5/5
213/213 [==============================] - 0s 2ms/step - loss: 0.9474 - accuracy: 0.6484
Test Accuracy: 0.6483984589576721
ok

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 1s 4ms/step - loss: 1.7197 - accuracy: 0.2610 - val_loss: 1.5007 - val_accuracy: 0.4074
Epoch 2/10
213/213 [==============================] - 1s 3ms/step - loss: 1.3631 - accuracy: 0.4153 - val_loss: 1.2385 - val_accuracy: 0.4276
Epoch 3/10
213/213 [==============================] - 1s 3ms/step - loss: 1.1717 - accuracy: 0.4407 - val_loss: 1.0977 - val_accuracy: 0.4738
Epoch 4/10
213/213 [==============================] - 1s 3ms/step - loss: 1.0543 - accuracy: 0.5060 - val_loss: 0.9998 - val_accuracy: 0.5645
Epoch 5/10
213/213 [==============================] - 1s 4ms/step - loss: 0.9673 - accuracy: 0.5959 - val_loss: 0.9236 - val_accuracy: 0.6435
Epoch 6/10
213/213 [==============================] - 1s 4ms/step - loss: 0.8974 - accuracy: 0.6866 - val_loss: 0.8612 - val_accuracy: 0.7200
Epoch 7/10
213/213 [==============================] - 1s 3ms/step - loss: 0.8389 - accuracy: 0.7433 - val_loss: 0.8083 - val_accuracy: 0.7605
Epoch 8/10
213/21

...

213/213 [==============================] - 2s 6ms/step - loss: 1.2244 - accuracy: 0.5584 - val_loss: 0.6705 - val_accuracy: 0.7955
Epoch 2/10
213/213 [==============================] - 1s 4ms/step - loss: 0.4730 - accuracy: 0.8475 - val_loss: 0.3442 - val_accuracy: 0.8942
Epoch 3/10
213/213 [==============================] - 1s 4ms/step - loss: 0.3032 - accuracy: 0.9008 - val_loss: 0.2755 - val_accuracy: 0.9080
Epoch 4/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2575 - accuracy: 0.9127 - val_loss: 0.2521 - val_accuracy: 0.9138
Epoch 5/10
213/213 [==============================] - 1s 6ms/step - loss: 0.2381 - accuracy: 0.9161 - val_loss: 0.2318 - val_accuracy: 0.9185
Epoch 6/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2218 - accuracy: 0.9209 - val_loss: 0.2213 - val_accuracy: 0.9208
Epoch 7/10
213/213 [==============================] - 1s 5ms/step - loss: 0.2154 - accuracy: 0.9218 - val_loss: 0.2168 - val_accuracy: 0.9224
Epoch 8/10
213/21

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 3s 8ms/step - loss: 1.1127 - accuracy: 0.6223 - val_loss: 0.5926 - val_accuracy: 0.7975
Epoch 2/25
213/213 [==============================] - 1s 4ms/step - loss: 0.4570 - accuracy: 0.8492 - val_loss: 0.3801 - val_accuracy: 0.8736
Epoch 3/25
213/213 [==============================] - 1s 4ms/step - loss: 0.3260 - accuracy: 0.8940 - val_loss: 0.2928 - val_accuracy: 0.9074
Epoch 4/25
213/213 [==============================] - 1s 5ms/step - loss: 0.2624 - accuracy: 0.9095 - val_loss: 0.2504 - val_accuracy: 0.9148
Epoch 5/25
213/213 [==============================] - 1s 6ms/step - loss: 0.2326 - accuracy: 0.9187 - val_loss: 0.2332 - val_accuracy: 0.9202
Epoch 6/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2222 - accuracy: 0.9184 - val_loss: 0.2285 - val_accuracy: 0.9205
Epoch 7/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2126 - accuracy: 0.9209 - val_loss: 0.2194 - val_accuracy: 0.9257
Epoch 8/25
213/21

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 2s 5ms/step - loss: 1.3081 - accuracy: 0.5032 - val_loss: 0.7492 - val_accuracy: 0.6891
Epoch 2/50
213/213 [==============================] - 1s 4ms/step - loss: 0.5278 - accuracy: 0.8082 - val_loss: 0.3524 - val_accuracy: 0.9011
Epoch 3/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2990 - accuracy: 0.9033 - val_loss: 0.2626 - val_accuracy: 0.9157
Epoch 4/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2463 - accuracy: 0.9162 - val_loss: 0.2360 - val_accuracy: 0.9171
Epoch 5/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2275 - accuracy: 0.9202 - val_loss: 0.2252 - val_accuracy: 0.9221
Epoch 6/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2179 - accuracy: 0.9229 - val_loss: 0.2196 - val_accuracy: 0.9229
Epoch 7/50
213/213 [==============================] - 1s 5ms/step - loss: 0.2118 - accuracy: 0.9271 - val_loss: 0.2188 - val_accuracy: 0.9220
Epoch 8/50
213/21

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Min accuracy: [[2], [2], [50], [0.2634440064430237]]

Max accuracy: [[16], [1], [50], [0.9315310120582581]]